# The Certified Developer Associate Exam
> My handcrafted notes

- toc: true
- badges: true
- categories: [certification, aws]

* Each region has multiple availability zones (usually 3, min is 2, max is 6)
    - each AZ is one or more discrete data centers with its own redundant power, networking and connectivity
    - AWS currently has 216 points of presence (205 Edge locations, 11 Regional caches) in 84 cities across 42 countries

# IAM

* IAM is a global service. Doesn't require Region selection
* A user can belong to multiple groups
* In AWS you apply the Least Privilege principle. i.e. you don't give more permissions than a user needs


## IAM Policy Structure

It consists of:
    
    - Version
    
    - Id: optional
    
    - Statement: one or more individual statements (required)
        
        * Sid: identifier for the statement (optional)
        * Effect: whether the statement allows or denies access (Allow, Deny)
        * Principal: account/user/role to which this policy applies
        * Action: List of actions this policy allows or denies
        * Resource: List of resources to which this action applies
        * Condition: conditions for when this policy is in effect (optional)
        

* Multi Factor Authentication (MFA) device options in AWS:

- Virtual MFA device (like Google Authenticator)
- Universal 2nd Factor (U2F) Security Key (A physical device)
- Hardware key Fob MFA device
- Hardware Key Fob MFA Device for AWS GovCloud (US)

* IAM roles are a secure way of granting permissions to entities you trust, to perform actions on your behalf. Examples of entities are:

    - IAM user in another account
    - Application code running on Ec2 that needs to perform actions on AWS resources
    - An AWS service that needs to act on resources in your account to provide features
    - Users from a corporate directory who use Identity Federation with SAML
    
* IAM security Tools:

    - IAM Credentials report (account-level)
        * This report lists all your account's users and status of their various credentials
    - IAM Access advisor shows the service permissions granted to each user and when they were last accessed

# EC2

* It is possible to bootstrap instances using an EC2 user data script. It allows you to run commands when your machine starts. That script is only run once at the instance first start. THE EC2 USER DATA SCRIPT RUNS WITH THE ROOT USER.

* Different EC2 instance types are optimised for different use cases. Naming convention:
    
    - m5.2xlarge, m: instance class, 5: generation, 2xlarge: size within the instance class

* Application ports to remember: 
    
    - 22: SSH
    - 21: FTP
    - 22: SFTP (Uploads using SSH)
    - 80: HTTP
    - 443: HTTPS
    - 3389: RDP (Remote Desktop Protocol) - log into Windows instances

* EBS Volume
    
    - It's a network drive
    - It can be detached from an EC2 instance and attached to another quickly
    - It's locked to an Availability Zone (AZ)
    - You can copy snapshots across AZ or region
    - **To move a volume across AZ, you'll need to snapshot it.**
    - An EBS volume cannot be attached to more than 1 EC2 instance at a time
    - However, an EC2 instance can have more than 1 EBS volumes attached to it at the same time
    - It is possible to leave EBS volumes unattached
    - By default, the root EBS volume is deleted
    - By default, any other attached EBS volume is not deleted
    - It's not necessary to detach a volume to do the snapshot, but it's recommended

* EC2 Instance Store

    - EBS volumes are network drives with good but limited performance
    - If you need a high-performance hardware disk, use EC2 instance store (It has high I/O performance)
    - EC2 instance store lose their storage when they're stopped (ephemeral)
    - EC2 instance store is therefore great for buffer/cache/scrath data/temporary content
    
* EBS volume types:

    - gp2/gp3 (SSD): General Purpose SSD volume that balance price and performance for a wide variety of workloads
    - io1/io2 (SSD): Highest-performance SSD volume for mission-critical low-latency or high-throughput workloads
    - st1 (HDD): Low cost HDD volume designed for frequently accessed, throughput-intensive workloads
    - sc2 (HDD): Lowest cost HDD volume designed for less frequently accessed workloads
    - Only gp2/gp3 and io1/io2 can be used as boot volumes
  
  
* EBS Multi Attach:

    - Attach the same EBS volume to multiple EC2 instances in the same AZ
    - Each instance has full read and write permissions to the volume
    - Use case: Achieve higher application availability in clustered Linux applications (ex - Teradata)
    - Must use a file system that is cluster-aware
    
* EFS - Elastic File System

    - Managed NFS that can be mounted on many EC2
    - EFS works with EC2 instances in multiple Availability Zones (AZ)
    - It's highly available, scalable and expensive (3x gp2), pay per use
    - Compatible with Linux based AMI only (Not Windows)
    - Encryption at rest using KMS
    - It has standard file API
    - Throughput 10GB+/s ... can grow to petabyte-scale network file system, automatically
    - Supports Performance (default) and Throughput mode
    - Storage Tiers (lifecycle management feature - move file after N days)
            - Standard: for frequently accessed files
            - Infrequently accessed files (EFS-IA): cost to retrieve files, lower price to store
            
* Load Balancer

    - Classic Load Balancer (v1 - old generation)
        - Supports HTTP, HTTPS, TCP, SSL
        - Health checks are either TCP based or HTTP based
        - You'll get a fixed hostname out of these
        - Supports only one SSL certificate
    - Application Load Balancer (v2 = new generation)
        - HTTP, HTTPS, WebSocket
        - It's a layer 7 load balancer only
        - Load balancing to multiple HTTP applications across machines (target groups)
        - Load balancing to multiple applications on the same machine
        - Support for HTTP/2 and websocket
        - Fixed hostname
        - The application servers don't see the IP of the client directly
            - The true IP of the client is inserted in the header X-Forwarded-For
            - We can also get Port (X-Forwarded-Port) and proto (X-Forwarded-Proto)
        - Supports multiple listeners with multiple SSL certificates. It uses Server Name Indication (SNI) to make it work.
        
    - Network Load Balancer (v2 - new generation)
        - TCP, TLS, UDP
        - Forward TCP & UDP traffic to your instances
        - Handle millions of requests per seconds
        - NLBs are used for extreme performance, TCP or UDP Traffic
        - You can assign 1 fixed IP (static IP) per availability zone to a NLB 
        - Supports multiple listeners with multiple SSL certificates. It uses Server Name Indication (SNI) to make it work.
        
    - Gateway Load Balancer
        - Operates at layer 3 (Network Layer)
        - Deploy, scale and manage a fleet of 3rd party network virtual appliances 
        - If you want all traffic to go through Firewalls, Intrusion Detection and prevention systems, Deep packet inspection, Payload manipulation before it reaches your application, then this LB is used
        - **Uses the GENEVE protocol on port 6081**
        
* Sticky sessions for a load balancer (Session Affinity)
    - It is possible to implement stickiness so that the same client is always redirected to the same instance behind a load balancer
    - This behaviour can be enabled for the CLASSIC Load Balancer and Application Load Balancer
    - The cookie used for stickiness has an expiration date you control
    - Use Case: Make sure the user doesn't lose the session data
    - The cookies can be
        - Application-based cookie (Custom cookie, generated by the target)
            - Can include custom attributes required by the application
            - Cookie name must be specified individually for each target group
            - DON'T use AWSALB, AWSALBAPP or AWSALBTG (reserved for use by the ELB)
        - Application Cookie
            - Generated by the Load Balancer
            - Cookie name is AWSALBAPP
        - Duration-based Cookie
            - Cookie generated by the load balancer
            - Cookie name is AWSALB for ALB and AWSELB for CLB
            

* Cross-Zone balancing
    - It's possible to distribute the traffic evenly across multiple AZ irrespective of how many instances there are within that AZ (with Cross-Zone balancing)
    - It's also possible to distribute the traffic based on the proportion of instances in each AZ (without Cross-Zone balancing)
    
    - Cross-Zone Load Balancing is Always On in an Application Load Balancer (can't be disabled. No charges of Inter AZ-data)
    - Cross-Zone Load Balancing is disabled by default in the Network Load Balancer. You pay charges for inter AZ data if enabled
    - Cross-Zone Load Balancing is disabled by default in the classic Load balancer (No charges for inter AZ data if enabled)

# SSL/TLS - Basics

- SSL = Secure Sockets Layer, used to encrypt connections
- TLS = Transport Layer Security, which is a newer version. Nowadays, TLS certificates are mainly used, but people still refer to them as SSL
- Public SSL certificates are issued by Certificate Authorities (CA)
- The load balancer used an X.509 certificate (or SSL/TLS certificate)
- You can manage certificates using AWS Certificate Manager

# SNI - Server Name Indication

- SNI solves the problem of loading multiple SSL certificates onto one web server (to serve multiple websites)
- It's a newer protocol and requires the client to indicate the hostname of the target server in the initial SSL handshake
- The server will then find the correct certificate, or return the default one
- It only works with Application Load Balancers (ALB) & Network Load Balancers (NLB) and CloudFront. It does NOT work with Classic Load Balancer (CLB)


# Connection Draining

- Known as Connection Draining - For Classic Load Balancer (CLB)
- Known as Deregistration Delay - For ALB and NLB
- The idea is that it will give some time to your instances to complete the In Flight Requests, while the instance is being deregisterd or marked unhealthy
- Once the instance is drained, it will stop sending new requests to the EC2 instance which is being de-registered